In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

23/05/28 17:55:18 WARN Utils: Your hostname, lucia-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/28 17:55:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/28 17:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/28 17:55:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [19]:
df = spark.read.json('201906_Usage_Bicimad.json')

In [20]:
df.show()
df.count()

+---------------+--------------------+--------+-----------+--------------+-------------+----------------+-----+-----------+--------------------+--------------------+---------+--------+
|_corrupt_record|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|track|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+---------------+--------------------+--------+-----------+--------------+-------------+----------------+-----+-----------+--------------------+--------------------+---------+--------+
|           null|{5cf83b752f3843a0...|       0|         21|            66|            8|              90| null|        219|{2019-06-01T00:00...|e4d55deb9ac172a8d...|        1|        |
|           null|{5cf83b762f3843a0...|       4|         19|           136|           19|              71| null|        359|{2019-06-01T00:00...|8a0c4123e924a50a9...|        1|   28039|
|           null|{5cf83b762f3843a0...|       4|         17|            38| 

450811

In [21]:
df = df.drop('user_day_code','zip_code','idplug_base','idunplug_base','track')
df.show()

+---------------+--------------------+--------+--------------+----------------+-----------+--------------------+---------+
|_corrupt_record|                 _id|ageRange|idplug_station|idunplug_station|travel_time|     unplug_hourTime|user_type|
+---------------+--------------------+--------+--------------+----------------+-----------+--------------------+---------+
|           null|{5cf83b752f3843a0...|       0|            66|              90|        219|{2019-06-01T00:00...|        1|
|           null|{5cf83b762f3843a0...|       4|           136|              71|        359|{2019-06-01T00:00...|        1|
|           null|{5cf83b762f3843a0...|       4|            38|              39|        375|{2019-06-01T00:00...|        1|
|           null|{5cf83b762f3843a0...|       5|            90|              66|        264|{2019-06-01T00:00...|        1|
|           null|{5cf83b762f3843a0...|       4|           166|             152|        367|{2019-06-01T00:00...|        1|
|           null

In [7]:
df1 = df.filter(df["user_type"]<=1 )
df1.show()
df1.count()

+---------------+--------------------+--------+--------------+----------------+-----------+---------+
|_corrupt_record|                 _id|ageRange|idplug_station|idunplug_station|travel_time|user_type|
+---------------+--------------------+--------+--------------+----------------+-----------+---------+
|           null|{5cf83b752f3843a0...|       0|            66|              90|        219|        1|
|           null|{5cf83b762f3843a0...|       4|           136|              71|        359|        1|
|           null|{5cf83b762f3843a0...|       4|            38|              39|        375|        1|
|           null|{5cf83b762f3843a0...|       5|            90|              66|        264|        1|
|           null|{5cf83b762f3843a0...|       4|           166|             152|        367|        1|
|           null|{5cf83b762f3843a0...|       5|            53|              55|        174|        1|
|           null|{5cf83b762f3843a0...|       0|           129|             133|   

397252

In [8]:
df2 = df1.filter(df["idplug_station"]!= df["idunplug_station"] )
df2.show()
df2.count() #descartamos los trayectos de una estación en si misma.

+---------------+--------------------+--------+--------------+----------------+-----------+---------+
|_corrupt_record|                 _id|ageRange|idplug_station|idunplug_station|travel_time|user_type|
+---------------+--------------------+--------+--------------+----------------+-----------+---------+
|           null|{5cf83b752f3843a0...|       0|            66|              90|        219|        1|
|           null|{5cf83b762f3843a0...|       4|           136|              71|        359|        1|
|           null|{5cf83b762f3843a0...|       4|            38|              39|        375|        1|
|           null|{5cf83b762f3843a0...|       5|            90|              66|        264|        1|
|           null|{5cf83b762f3843a0...|       4|           166|             152|        367|        1|
|           null|{5cf83b762f3843a0...|       5|            53|              55|        174|        1|
|           null|{5cf83b762f3843a0...|       0|           129|             133|   

388817

In [9]:
df3 = df2.filter(df["idplug_station"]== df["idunplug_station"] )
df3.show()
df3.count()#comprobamos que efectivamente no hay trayectos de una estación en si misma

+---------------+---+--------+--------------+----------------+-----------+---------+
|_corrupt_record|_id|ageRange|idplug_station|idunplug_station|travel_time|user_type|
+---------------+---+--------+--------------+----------------+-----------+---------+
+---------------+---+--------+--------------+----------------+-----------+---------+



0

In [10]:
df2.groupBy('ageRange').count().show()


+--------+------+
|ageRange| count|
+--------+------+
|       0|172485|
|       6|  2167|
|       5| 79649|
|       1|   537|
|       3| 20328|
|       2|  1786|
|       4|111865|
+--------+------+



In [28]:
from pyspark.sql.functions import collect_list, struct
dfg = df2.groupBy('_id').\
    agg(collect_list(struct('idunplug_station','idplug_station')).alias('trayectos_1sentido'))

dfg.show()

#realmente necesitamos que en esta primera columna aparezcan los trayectos de froma que la primera coordenada sea la 
#más pequeña y la segunda la más grande para que cuando hagamos el groupBy no distinga el sentido.

+--------------------+------------------+
|                 _id|trayectos_1sentido|
+--------------------+------------------+
|{5cf83b752f3843a0...|        [{90, 66}]|
|{5cf83b762f3843a0...|       [{71, 136}]|
|{5cf83b762f3843a0...|        [{39, 38}]|
|{5cf83b762f3843a0...|        [{66, 90}]|
|{5cf83b762f3843a0...|      [{152, 166}]|
|{5cf83b762f3843a0...|        [{55, 53}]|
|{5cf83b762f3843a0...|      [{133, 129}]|
|{5cf83b762f3843a0...|      [{153, 169}]|
|{5cf83b762f3843a0...|       [{44, 129}]|
|{5cf83b762f3843a0...|       [{85, 133}]|
|{5cf83b762f3843a0...|        [{31, 83}]|
|{5cf83b762f3843a0...|        [{38, 53}]|
|{5cf83b762f3843a0...|       [{118, 13}]|
|{5cf83b762f3843a0...|        [{27, 10}]|
|{5cf83b762f3843a0...|        [{90, 74}]|
|{5cf83b762f3843a0...|       [{100, 54}]|
|{5cf83b762f3843a0...|        [{67, 83}]|
|{5cf83b762f3843a0...|        [{67, 83}]|
|{5cf83b762f3843a0...|       [{63, 163}]|
|{5cf83b762f3843a0...|        [{42, 84}]|
+--------------------+------------

In [13]:
import pandas
dfg1 = dfg.groupBy('trayectos_1sentido').count().pandas_api()

/home/lucia/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [14]:
dfgorder = dfg1.nlargest(int(.01*len(dfg1.index)),["count"])

In [15]:
dfgorder

,trayectos_1sentido,count
20068,"[(9, 149)]",299
7903,"[(49, 135)]",228
4054,"[(135, 132)]",219
16734,"[(135, 49)]",204
25388,"[(169, 149)]",201
6353,"[(163, 149)]",188
7777,"[(149, 9)]",184
19479,"[(132, 135)]",182
1870,"[(149, 130)]",181
13998,"[(149, 169)]",179


In [16]:
len(dfgorder)

284